In [1]:
%load_ext Cython

In [2]:
from os import listdir,getcwd
from os.path import isfile, join
import re

def CountCollector(cxtName):
    onlyfiles = [f for f in listdir(getcwd()) if isfile(join(getcwd(), f))]
    dic={}
    fcnt=0
    print(len(onlyfiles))

    for f in onlyfiles:
           if f[:len(cxtName)]==cxtName:
                print(f)
                result1 = re.search(r"b_\[([0-9_]+),",f)
                result2 = re.search(r", (-?[0-9_]+)\]",f)
                result3 = re.search(r"c_([0-9_]+)_",f)
                if result1 and result2 and result3:
                    S=int(result1.group(1))
                    E=int(result2.group(1))
                    cnt=int(result3.group(1))
                    dic[(S,E)]=cnt
                    fcnt+=1
    print("Total branches:",fcnt)
    return dic

def PartialCountCollector(cxtName,prefix):
    onlyfiles = [f for f in listdir(getcwd()) if isfile(join(getcwd(), f))]
    dic={}
    fcnt=0
    print(len(onlyfiles))

    for f in onlyfiles:
           if f[:len(cxtName)+len(prefix)]==prefix+cxtName:
                print(f)
                result1 = re.search(r"b_\[([0-9_]+),",f)
                result2 = re.search(r", (-?[0-9_]+)\]",f)
                if result1 and result2:
                    S=int(result1.group(1))
                    E=int(result2.group(1))
                    with open(prefix+cxtName+"_b_["+str(S)+", "+str(E)+"]_log.txt") as f:
                        for line in f:
                            pass
                        last_line = line
                    A,cnt=last_line.strip().split(" ")                            
                    dic[(S,E)]=(int(A),int(cnt))
                    fcnt+=1
    print("Total branches:",fcnt)
    return dic

In [3]:
%%cython --annotate

def lst2int(lst):
    r=0
    for i in lst:
        r |= (1<<i)
    return r  

class nnBitContext:
    def __init__(self, n=3, i=0, m=3):
            self.attr_names=[str(m+1) for m in range(n)]
            self.objects=[]
            #print(bin(i))
            for j in range(m):                
                self.objects.append(i & ((1<<n) - 1))
                #print(bin(i & ((1<<n) - 1)))
                i=i>>n
                #print(bin(i))
            self.nAttr=len(self.attr_names)
            self.nObj=len(self.objects)
            self.name="Context"

      
 
    def aClosure(self,aSet):
        #вычисление признакового замыкания
        B=(1<<self.nAttr)-1
        for g in self.objects:
            if aSet & g == aSet:
                B=B & g        
        return B

    
def cxtFromFile(filename):
    cxt=nnBitContext(0,0,0)
   
    f = open(filename, "r")
    print(f.readline())
    print(f.readline())
    
   
    cxt.nObj=int(f.readline())
    cxt.nAttr=int(f.readline())
    
    print(f.readline())
    
    for i in range(cxt.nObj):
        print(f.readline())
    
    for i in range(cxt.nAttr):
        cxt.attr_names.append(f.readline())
    

    for l in range(cxt.nObj):
        o=f.readline().replace('.','0').replace('X','1')
        print(o)
        
        cxt.objects.append(int(o,2))
        #print(bin(i & ((1<<n) - 1)))
        
        #print(bin(i)
    cxt.name=filename[:-4]
    f.close()
    return cxt         
    
def BitNextClosure(A,bCxt):
    #print("Bit NextClosure")
    for m in range(bCxt.nAttr-1,-1,-1):
        #print("m=",m)
        if A & (1<<m):
            #print("A=",A)
            A= A & ~(1<<m)
            #print("A= A & ~(1<<m)",A)
        else:
            B=bCxt.aClosure(A |(1<<m))
            #print("A=",A,bin(A))
            #print("B=cl(AUm)",B)
            B_A=B & ~A
            #print("B-A",B_A,bin(B_A))
            if not B_A & ((1<<m)-1):
                #print("B_A", B_A, bin(B_A), "((1<<m)-1)", ((1<<m)-1), bin(((1<<m)-1)))  
                return B
    print("Branch terminated with -1")
    return -1  
    
    
    

def FirstClosure(bCxt):
    #print("First closure")
    if bCxt.objects==[]:
        return 2**bCxt.nAttr-1
    eCl=bCxt.objects[0]
    if eCl:
        for g in bCxt.objects[1:]:
            eCl= eCl & g 
  
    return eCl

def AllClosures(bCxt):
    #print("All closures")
    L=[]
    A=FirstClosure(bCxt)
    #print("First Closure",A,bin(A))
    while A!=-1:
        #print ("A=",A, "binary representation", bin(A))
        L.append(A)
        A=BitNextClosure(A,bCxt)
    return L

def CountAllClosures(bCxt):
    
    #print("All closures")
    cnt=0
    prevcnt=0
    A=FirstClosure(bCxt)
    #print("First Closure",A,bin(A))
    while A!=-1:
        #print ("A=",A, "binary representation", bin(A))
        cnt+=1
        A=BitNextClosure(A,bCxt)
        if cnt%1000000==0: 
            if cnt!=prevcnt: 
                print(cnt)
                prevcnt=cnt
    return cnt


def GenerateMilestones(bCxt):
    M=[2**i-1 for i in range(bCxt.nAttr)]
    MS=[]
    Pairs=[]
    L=[BitNextClosure(m,bCxt) for m in M]
    for i in range(len(L)):
        if L[i]==2**bCxt.nAttr-1:
            print(i,"goes to 2^",bCxt.nAttr-1)
        if L[i]==2**i-1:
            print(2**i-1," is closed")
    
    MS.append(FirstClosure(bCxt))
    

    
    for i in range(len(L)):
        if L[i] not in MS:
            MS.append(L[i])
            
    for i in range(len(MS)):
        if i+1<len(MS): Pairs.append(tuple([MS[i],MS[i+1]]))
            
    print("i",i)        
        
    if MS[-1]!=-1:
        Pairs.append(tuple([MS[-1],-1]))
        
    return Pairs

def CountParClosures(bCxt,S,E):
    cnt=0
    prevcnt=0
    A=S

    while A!=E:
        #print ("A=",A, "binary representation", bin(A))
        cnt+=1
        
        if cnt%100000000==0: 
            if cnt!=prevcnt: 
                print(A," ",cnt)
                prevcnt=cnt
                f = open("temp_"+bCxt.name+"_b_"+str([S,E])+"_log.txt", 'a')
                f.write(str(A)+" "+str(cnt)+"\n")
                f.close()
        A=BitNextClosure(A,bCxt)
    print("Branch ",[S,E]," is terminated with count=",cnt)
    f = open(bCxt.name+"_b_"+str([S,E])+"_c_"+str(cnt)+"_log.txt", 'w')
    f.write(str([S,E])+" "+str(cnt))
    f.close()
    
    return cnt


def CountParClosures2(bCxt,S,E,A,cnt):
    prevcnt=cnt
    A=BitNextClosure(A,bCxt)

    while A!=E:
        #print ("A=",A, "binary representation", bin(A))
        cnt+=1
        
        if cnt%1000000==0: 
            if cnt!=prevcnt: 
                #print(A," ",cnt)
                prevcnt=cnt
                f = open("corr_temp_"+bCxt.name+"_b_"+str([S,E])+"_log.txt", 'a')
                f.write(str(A)+" "+str(cnt)+"\n")
                f.close()
        A=BitNextClosure(A,bCxt)
    print("Branch ",[S,E]," is terminated with count=",cnt)
    f = open(bCxt.name+"_b_"+str([S,E])+"_c_"+str(cnt)+"_log.txt", 'w')
    f.write(str([S,E])+" "+str(cnt))
    f.close()
    
    return cnt





In [8]:
from tqdm.auto import tqdm

from cython.parallel import prange
from time import time

In [10]:
%%time


Cxt9=cxtFromFile("Generated_AL5_alpha.cxt")

if __name__ == "__main__":
        
    Pairs=GenerateMilestones(Cxt9)
    
    dColl=CountCollector("Generated_AL5_alpha")
    
    
    dParColl=PartialCountCollector("Generated_AL5_alpha","temp_")
    
    print(len(Pairs))

    parList=[]
    
    cnt=0
    
  
    for p in Pairs:
        if p not in dColl:
            if p in dParColl:
                pass
            else: 
                print("missing pair:",p) 
                parList.append([Cxt9,p[0],p[1]])
                

    print(parList)
    print(len(parList))
       
     
    

    
    
    rCxt9 = [CountParClosures(*parList[i]) for i in tqdm(prange(len(parList), num_threads=8, nogil=True))]
    
    print(len(rCxt9))
    print("total precomuted:",sum(dColl.values()))
    print("total postcomputed:",sum(rCxt9))
    print("total computed:",sum(rCxt9)+sum(dColl.values()))



 

B





ab

ac

bc

abc

ad

bd

abd

cd

acd

bcd

abcd

ae

be

abe

ce

ace

bce

abce

de

ade

bde

abde

cde

acde

bcde

0001111111111111111111111111111111111111111111111111111

1110001111111111111111111111111111111111111111111111111

1111110001111111111111111111111111111111111111111111111

1001001000011111111111111111111111111111111111111111111

1111111111100011111111111111111111111111111111111111111

1111111111111100011111111111111111111111111111111111111

0101111111110010000111111111111111111111111111111111111

1111111111111111111000111111111111111111111111111111111

1110101111101011111100001111111111111111111111111111111

1111110101111101011010110011111111111111111111111111111

1101101101011011010110101001111111111111111111111111111

1111111111111111111111111110001111111111111111111111111

1111111111111111111111111111110001111111111111111111111

0011111111111111111111111111001000011111111111111111111

1111111111111111111111111111111111100011111111111111111

111001111111111111

  0%|          | 0/45 [00:00<?, ?it/s]

Branch  [0, 268435456]  is terminated with count= 1
Branch  [268435456, 1073741825]  is terminated with count= 169146
Branch  [1073741825, 4294967299]  is terminated with count= 140185
Branch  [4294967299, 15]  is terminated with count= 100274
Branch  [15, 4294967311]  is terminated with count= 1
Branch  [4294967311, 137438953503]  is terminated with count= 88342
Branch  [137438953503, 127]  is terminated with count= 57492
Branch  [127, 137438953599]  is terminated with count= 1
Branch  [137438953599, 511]  is terminated with count= 46349
Branch  [511, 137438953983]  is terminated with count= 1
Branch  [137438953983, 4095]  is terminated with count= 50366
Branch  [4095, 137438957567]  is terminated with count= 1
Branch  [137438957567, 35184372097023]  is terminated with count= 18743
Branch  [35184372097023, 32767]  is terminated with count= 9727
Branch  [32767, 35184372121599]  is terminated with count= 1
Branch  [35184372121599, 131071]  is terminated with count= 7527
Branch  [131071,